# 🧪 Panel Chemistry - Molstar

The Panel Chemistry `MolStarViwer` allows you to use the powerful Mol* ('MolStar') viewer.

![MolStarViewer](../assets/Molstar.png)


Interaction from python is currently limited to loading structures from the PDB/EMDB/Alphafold DB and toggling controls, logs, expanded, etc on and off.
See the [MolStar Viewer](https://molstar.org/viewer-docs/) for more information on how to use the viewer.


When using Mol*, please cite:

- David Sehnal, Sebastian Bittrich, Mandar Deshpande, Radka Svobodová, Karel Berka, 
Václav Bazgier, Sameer Velankar, Stephen K Burley, Jaroslav Koča, Alexander S Rose: 
Mol* Viewer: modern web app for 3D visualization and analysis of large biomolecular structures, 
Nucleic Acids Research, 2021; https://doi.org/10.1093/nar/gkab314.



In [1]:
from panel_chemistry.pane import MolStarViewer
import panel as pn
import param

In [2]:
pn.extension(sizing_mode="stretch_width", template="fast")

In [3]:
MOLSTAR_KWARGS = dict(width=1000, height=600)


class MolStarApp(param.Parameterized):

    database = param.Selector(default='PDB', objects=['PDB', 'EMDB', 'Alphafold'])

    pdb_id = param.String(default=None, doc="PDB ID to load", label="PDB ID")

    emdb_id = param.String(default=None, doc="EMDB ID to load", label="EMDB ID")

    emdb_detail = param.Selector(default=6, objects=list(range(1, 12)))

    alphafold_id = param.String(default=None, doc="Alphafold ID to load", label="Alphafold ID")

    layout_is_expanded = param.Boolean(default=False, doc="Expand the layout")

    layout_show_controls = param.Boolean(default=False, doc="Show the controls")

    layout_show_remote_state = param.Boolean(default=False, doc="Show the remote state")

    layout_show_sequence = param.Boolean(default=True, doc="Show the sequence")

    layout_show_log = param.Boolean(default=True, doc="Show the log")

    layout_show_left_panel = param.Boolean(default=True, doc="Show the left panel")

    viewport_show_expand = param.Boolean(default=True, doc='Show the expand button')

    viewport_show_selection_mode = param.Boolean(default=False, doc='Show the selection mode')

    viewport_show_animation = param.Boolean(default=False)

    rerender = param.Action(lambda self: self._action_rerender())

    def __init__(self, **params):
        super().__init__(**params)
        self.widgets = pn.Param(self, show_name=False)._widgets

        self.controls = pn.Column(*self.control_widgets, width=250)
        self.molstar = pn.Column(MolStarViewer(**MOLSTAR_KWARGS))
        self.app = pn.Row(
            self.controls, self.molstar
        )

    @property
    def control_widgets(self):
        excluded = {'pdb_id', 'emdb_id', 'emdb_detail', 'alphafold_id'}
        if self.database == 'PDB':
            excluded -= {'pdb_id'}
        elif self.database == 'EMDB':
            excluded -= {'emdb_id', 'emdb_detail'}
        elif self.database == 'Alphafold':
            excluded -= {'alphafold_id'}

        return [w for k, w in self.widgets.items() if k not in excluded]

    @param.depends('database', watch=True)
    def _on_database_change(self):
        self.controls[:] = self.control_widgets

    def _action_rerender(self):
        ms = self.molstar[0]
        params = {p for p in ms.param}
        options = params & self.widgets.keys()

        kwargs = {opt: getattr(self, opt) for opt in options}

        if self.database == 'PDB':
            kwargs['pdb'] = self.pdb_id
        elif self.database == 'EMDB':
            kwargs['emdb'] = (self.emdb_id, {'detail': self.emdb_detail})
        elif self.database == 'Alphafold':
            kwargs['alphafold_db'] = self.alphafold_id

        viewer = MolStarViewer(**kwargs, **MOLSTAR_KWARGS)
        self.molstar[:] = [viewer]


molstar = MolStarApp()
molstar.app


Row(sizing_mode='stretch_width')
    [0] Column(width=250)
        [0] Select(name='Database', options=OrderedDict([('PDB', ...]), sizing_mode='stretch_width', value='PDB')
        [1] TextInput(name='PDB ID', sizing_mode='stretch_width')
        [2] Checkbox(name='Layout is expanded', sizing_mode='stretch_width')
        [3] Checkbox(name='Layout show controls', sizing_mode='stretch_width')
        [4] Checkbox(name='Layout show r..., sizing_mode='stretch_width')
        [5] Checkbox(name='Layout show sequence', sizing_mode='stretch_width', value=True)
        [6] Checkbox(name='Layout show log', sizing_mode='stretch_width', value=True)
        [7] Checkbox(name='Layout show left panel', sizing_mode='stretch_width', value=True)
        [8] Checkbox(name='Viewport show expand', sizing_mode='stretch_width', value=True)
        [9] Checkbox(name='Viewport show s..., sizing_mode='stretch_width')
        [10] Checkbox(name='Viewport show animation', sizing_mode='stretch_width')
        [11] Button(name='Rerender', sizing_mode='stretch_width')
    [1] Column(sizing_mode='stretch_width')
        [0] MolStarViewer(height=600, sizing_mode='fixed', width=1000)